In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import warnings
import imageio
from skimage.transform import resize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings("ignore")

In [15]:
label_df = pd.read_csv('label.csv')
len(label_df)

191229

In [7]:
fake_train_dir = '//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLearning/Fake_Stanford_256_final/'
fake_img_list = os.listdir(fake_train_dir)
len(fake_img_list)

#### super nice sort algorithm! ############
import re

digits = re.compile(r'(\d+)')
def tokenize(filename):
    return tuple(int(token) if match else token
                 for token, match in
                 ((fragment, digits.search(fragment))
                  for fragment in digits.split(filename)))

# Now you can sort your PDF file names like so:
fake_img_list.sort(key=tokenize)

191229

In [16]:
label_df.insert(loc=0, column='Fake Image Index', value=fake_img_list)
Data_Dir = '/media/tianyu.han/mri-scratch/DeepLearning/Fake_Stanford_256_2/'
label_df['Image Path'] = label_df['Fake Image Index'].map(lambda x: Data_Dir + x)

In [5]:
nih_df = pd.read_csv('fake_nih_78468.csv')
nih_train_df, nih_validtest_df = [x for _, x in nih_df.groupby(nih_df['fold'] != 'train')]
nih_train_df = nih_train_df.reset_index(drop=True)
len(nih_train_df)
len(nih_validtest_df)

78468

33652

In [6]:
nih_train_df.head()
nih_validtest_df.head()

,Image Index,Cardiomegaly,Effusion,Atelectasis,Pneumothorax,Pneumonia,Edema,Consolidation,fold,Image Path
0,fake_0.png,1,0,0,0,0,0,0,train,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...
1,fake_1.png,1,0,0,0,0,0,0,train,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...
2,fake_2.png,1,1,0,0,0,0,0,train,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...
3,fake_3.png,0,0,0,0,0,0,0,train,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...
4,fake_4.png,0,0,0,0,0,0,0,train,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...


,Image Index,Cardiomegaly,Effusion,Atelectasis,Pneumothorax,Pneumonia,Edema,Consolidation,fold,Image Path
78468,00000010_000.png,0,0,0,0,0,0,0,test,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...
78469,00000012_000.png,0,1,0,0,0,0,0,val,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...
78470,00000015_000.png,0,0,0,0,0,0,0,test,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...
78471,00000020_000.png,0,0,0,0,0,0,0,test,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...
78472,00000020_001.png,0,0,0,0,0,0,0,test,//nas5.pmi.rwth-aachen.de/mri-scratch/DeepLear...


In [17]:
label_df = label_df.drop(columns=['Fake Image Index', 'Path', 'No Finding', 'Enlarged Cardiomediastinum',
                                 'Lung Lesion', 'Pleural Other', 'Fracture', 'Image Index', 
                                  'Support Devices'])
label_df.insert(loc=0, column='Image Index', value=fake_img_list)
label_df.insert(loc=8, column='fold', value='train')
label_df.rename({'Pleural Effusion': 'Effusion'}, axis=1, inplace=True)
label_df

,Image Index,Cardiomegaly,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,fold,Effusion,Image Path
0,fake_0.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
1,fake_1.png,1.0,1.0,1.0,0.0,0.0,1.0,0.0,train,1.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
2,fake_2.png,0.0,1.0,0.0,0.0,0.0,0.0,0.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
3,fake_3.png,0.0,0.0,1.0,0.0,0.0,0.0,0.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
4,fake_4.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
5,fake_5.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
6,fake_6.png,0.0,0.0,0.0,0.0,0.0,0.0,1.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
7,fake_7.png,0.0,0.0,0.0,0.0,0.0,0.0,1.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
8,fake_8.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
9,fake_9.png,1.0,1.0,0.0,0.0,0.0,1.0,1.0,train,0.0,/media/tianyu.han/mri-scratch/DeepLearning/Fak...


In [8]:
nih_column = nih_df.columns.tolist()
label_df = label_df[nih_column]
label_df.head()

,Image Index,Cardiomegaly,Effusion,Atelectasis,Pneumothorax,Pneumonia,Edema,Consolidation,fold,Image Path
0,fake_0.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
1,fake_1.png,1.0,1.0,1.0,0.0,0.0,1.0,0.0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
2,fake_2.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
3,fake_3.png,0.0,0.0,0.0,0.0,0.0,1.0,0.0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
4,fake_4.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...


In [9]:
validtest_img_dir = '/media/tianyu.han/mri-scratch/DeepLearning/NIH_Chest/images/'
train_img_dir = '/media/tianyu.han/mri-scratch/DeepLearning/Fake_78468/'

In [10]:
nih_train_df = nih_train_df.drop(columns='Image Path')
nih_validtest_df = nih_validtest_df.drop(columns='Image Path')
nih_train_df['Image Path'] = nih_train_df['Image Index'].map(lambda x: train_img_dir + x)
nih_validtest_df['Image Path'] = nih_validtest_df['Image Index'].map(lambda x: validtest_img_dir + x)

In [11]:
nih_train_df.head()
nih_validtest_df.head()

,Image Index,Cardiomegaly,Effusion,Atelectasis,Pneumothorax,Pneumonia,Edema,Consolidation,fold,Image Path
0,fake_0.png,1,0,0,0,0,0,0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
1,fake_1.png,1,0,0,0,0,0,0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
2,fake_2.png,1,1,0,0,0,0,0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
3,fake_3.png,0,0,0,0,0,0,0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...
4,fake_4.png,0,0,0,0,0,0,0,train,/media/tianyu.han/mri-scratch/DeepLearning/Fak...


,Image Index,Cardiomegaly,Effusion,Atelectasis,Pneumothorax,Pneumonia,Edema,Consolidation,fold,Image Path
78468,00000010_000.png,0,0,0,0,0,0,0,test,/media/tianyu.han/mri-scratch/DeepLearning/NIH...
78469,00000012_000.png,0,1,0,0,0,0,0,val,/media/tianyu.han/mri-scratch/DeepLearning/NIH...
78470,00000015_000.png,0,0,0,0,0,0,0,test,/media/tianyu.han/mri-scratch/DeepLearning/NIH...
78471,00000020_000.png,0,0,0,0,0,0,0,test,/media/tianyu.han/mri-scratch/DeepLearning/NIH...
78472,00000020_001.png,0,0,0,0,0,0,0,test,/media/tianyu.han/mri-scratch/DeepLearning/NIH...


In [12]:
frames = [label_df, nih_train_df, nih_validtest_df]
nih_stanford_df = pd.concat(frames)
nih_stanford_df = nih_stanford_df.reset_index(drop=True)
len(nih_stanford_df)

303349

In [13]:
nih_stanford_df.tail()

,Image Index,Cardiomegaly,Effusion,Atelectasis,Pneumothorax,Pneumonia,Edema,Consolidation,fold,Image Path
303344,00030790_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test,/media/tianyu.han/mri-scratch/DeepLearning/NIH...
303345,00030792_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test,/media/tianyu.han/mri-scratch/DeepLearning/NIH...
303346,00030792_001.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test,/media/tianyu.han/mri-scratch/DeepLearning/NIH...
303347,00030797_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,test,/media/tianyu.han/mri-scratch/DeepLearning/NIH...
303348,00030802_000.png,0.0,0.0,0.0,0.0,0.0,0.0,0.0,val,/media/tianyu.han/mri-scratch/DeepLearning/NIH...


In [ ]:
saveDir = '/media/tianyu.han/mri-scratch/DeepLearning/CheXpert_Dataset/DL_DENSENET/Densenet_Stanford_lr/'
file = 'nih_stanford.csv'
nih_stanford_df.to_csv(os.path.join(saveDir, file), index=False)